# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import matplotlib.pyplot as plt

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from ate_models.ATE_BERT_Dropout_CNN_BiLSTM_Linear import ATE_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_cnn_bilstm_linear_512.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_cnn_bilstm_linear_512.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []
    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses

    outputs, targets = validation(model, validation_dataloader)

    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ATE/SemEval16 - Task 5 - Restaurants/plots/bert_pt_do_cnn_bilstm_lin/train_loss_run_{i + 1}.png')

    plt.clf()
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1140022277832031
Epoch: 0/2, Batch: 50/501, Loss: 0.5221658945083618
Epoch: 0/2, Batch: 100/501, Loss: 0.15322467684745789
Epoch: 0/2, Batch: 150/501, Loss: 0.08344922214746475
Epoch: 0/2, Batch: 200/501, Loss: 0.051482122391462326
Epoch: 0/2, Batch: 250/501, Loss: 0.030551783740520477
Epoch: 0/2, Batch: 300/501, Loss: 0.03515926003456116
Epoch: 0/2, Batch: 350/501, Loss: 0.030131492763757706
Epoch: 0/2, Batch: 400/501, Loss: 0.044823456555604935
Epoch: 0/2, Batch: 450/501, Loss: 0.02224038727581501
Epoch: 0/2, Batch: 500/501, Loss: 0.03260359168052673
Epoch: 1/2, Batch: 0/501, Loss: 0.03461591899394989
Epoch: 1/2, Batch: 50/501, Loss: 0.05116621032357216
Epoch: 1/2, Batch: 100/501, Loss: 0.061228424310684204
Epoch: 1/2, Batch: 150/501, Loss: 0.014008823782205582
Epoch: 1/2, Batch: 200/501, Loss: 0.03966030105948448
Epoch: 1/2, Batch: 250/501, Loss: 0.03220851719379425
Epoch: 1/2, Batch: 300/501, Loss: 0.030350349843502045
Epoch: 1/2, Batch: 3

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.10434889793396
Epoch: 0/2, Batch: 50/501, Loss: 0.3326457738876343
Epoch: 0/2, Batch: 100/501, Loss: 0.10724114626646042
Epoch: 0/2, Batch: 150/501, Loss: 0.05139080807566643
Epoch: 0/2, Batch: 200/501, Loss: 0.0411040224134922
Epoch: 0/2, Batch: 250/501, Loss: 0.04565957188606262
Epoch: 0/2, Batch: 300/501, Loss: 0.0314297191798687
Epoch: 0/2, Batch: 350/501, Loss: 0.023328833281993866
Epoch: 0/2, Batch: 400/501, Loss: 0.012339103035628796
Epoch: 0/2, Batch: 450/501, Loss: 0.030100708827376366
Epoch: 0/2, Batch: 500/501, Loss: 0.02282460778951645
Epoch: 1/2, Batch: 0/501, Loss: 0.0181474220007658
Epoch: 1/2, Batch: 50/501, Loss: 0.012101374566555023
Epoch: 1/2, Batch: 100/501, Loss: 0.018924085423350334
Epoch: 1/2, Batch: 150/501, Loss: 0.025568518787622452
Epoch: 1/2, Batch: 200/501, Loss: 0.015688953921198845
Epoch: 1/2, Batch: 250/501, Loss: 0.013581948354840279
Epoch: 1/2, Batch: 300/501, Loss: 0.012071270495653152
Epoch: 1/2, Batch: 350/

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.108407974243164
Epoch: 0/2, Batch: 50/501, Loss: 0.5189666152000427
Epoch: 0/2, Batch: 100/501, Loss: 0.14142046868801117
Epoch: 0/2, Batch: 150/501, Loss: 0.08267635852098465
Epoch: 0/2, Batch: 200/501, Loss: 0.040450308471918106
Epoch: 0/2, Batch: 250/501, Loss: 0.03132370859384537
Epoch: 0/2, Batch: 300/501, Loss: 0.022990979254245758
Epoch: 0/2, Batch: 350/501, Loss: 0.031113281846046448
Epoch: 0/2, Batch: 400/501, Loss: 0.020561091601848602
Epoch: 0/2, Batch: 450/501, Loss: 0.013209493830800056
Epoch: 0/2, Batch: 500/501, Loss: 0.02954140119254589
Epoch: 1/2, Batch: 0/501, Loss: 0.029649389907717705
Epoch: 1/2, Batch: 50/501, Loss: 0.02405465580523014
Epoch: 1/2, Batch: 100/501, Loss: 0.023307012394070625
Epoch: 1/2, Batch: 150/501, Loss: 0.01490751001983881
Epoch: 1/2, Batch: 200/501, Loss: 0.023895015940070152
Epoch: 1/2, Batch: 250/501, Loss: 0.027554700151085854
Epoch: 1/2, Batch: 300/501, Loss: 0.015118242241442204
Epoch: 1/2, Batch:

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.995078,0.995078,0.526929,0.995078,0.363465,0.995078,0.383117,415.784072
1,0.995219,0.995219,0.454790,0.995219,0.370154,0.995219,0.389278,383.933037
2,0.994910,0.994910,0.549152,0.994910,0.352124,0.994910,0.365240,384.568043
3,0.994887,0.994887,0.496924,0.994887,0.350833,0.994887,0.364275,385.595846
4,0.995324,0.995324,0.700308,0.995324,0.356688,0.995324,0.376474,381.474509
5,0.994906,0.994906,0.446301,0.994906,0.358998,0.994906,0.374490,373.427891
6,0.994996,0.994996,0.582448,0.994996,0.362355,0.994996,0.382391,349.605191
7,0.994906,0.994906,0.434815,0.994906,0.357065,0.994906,0.371125,345.543205
8,0.994934,0.994934,0.610850,0.994934,0.351595,0.994934,0.365276,348.022727
9,0.995004,0.995004,0.423372,0.995004,0.342627,0.995004,0.349408,351.684240


In [20]:
results.to_csv(STATS_OUTPUT)